# Structure MIMI Reports with ChatGPT
This is an example notebook on how GPT-4 could be used to extract information 
and structure the over 200k chest x-ray reports in the MIMIC database. 

In [ ]:
import sys
sys.path.append("..")

In [ ]:
from pathlib import Path
from gpt import GPTStructuredReporting
from tqdm import tqdm
import json
import openai
import time

In [ ]:
openai.api_key_path = "path/to/your/openai-key"
mimic_reports = "path/to/reports"

In [ ]:
with open("../static/report_templates.json", "r") as f: 
    template = json.loads(f.read())["XRAY_CHEST"]

We use one of our templates, but feel free to provide your own template in the prompt. 

In [ ]:
system = (
    "This is a JSON template for a structured report for a chest x-ray."   
    "Fill out the template with the information form the unstructured radiology "
    "report the user will provide you. If the finding is present enter 1 if it is absent enter 0. "
    "Here is the template:\n") + json.dumps(template)

In [ ]:
def gpt_struct_report(report_text): 
    response = openai.ChatCompletion.create(
            model="gpt-4", # gpt-3.5-turbo would also be enough and 10-50 times cheaper
            messages=[
                {"role": "system", "content": system},
                {"role": "user", "content": str(report_text)},
            ]
    ) 
    structured_report = json.loads(response["choices"][0]["message"]["content"])
    return structured_report

In [ ]:
for report in tqdm(list(mimic_reports)): 
    out_name = Path("structured_mimic_reports")/report.name.replace(".txt", ".json")
    for i in range(5):  # number of times until timeout
        try: 
            if not out_name.exists(): 
                structured_report = gpt_struct_report(report)
                with open(out_name, "w+") as f: 
                    f.write(json.dumps(structured_report))
        except Exception as e:
            print(e)  # openai api frequently has errors due to too many requests
            print("trying again in 5 seconds")
            if i < 4:  # no sleep after last try
                time.sleep(5)